# Web scrapping of flight price prediction

In [1]:
#Import necessory libraries
import selenium
import pandas as pd
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,ElementClickInterceptedException, ElementNotVisibleException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


In [2]:
# Activating webdriver

driver=webdriver.Chrome(r"C:\Users\Lenovo\Downloads\Compressed\chromedriver_win32\chromedriver.exe")



In [3]:
driver.get("https://flight.yatra.com/air-search-ui/dom2/trigger?type=O&viewName=normal&flexi=0&noOfSegments=1&origin=DEL&originCountry=IN&destination=BOM&destinationCountry=IN&flight_depart_date=23%2F02%2F2022&ADT=1&CHD=0&INF=0&class=Economy&source=fresco-home&unqvaldesktop=509417007759")
time.sleep(35)

In [4]:
#making list of locations
city_lst = ['New Delhi', 'Mumbai', 'Bangalore', 'Chennai','Hyderabad ','Goa ','Kolkata ','Jaipur ','Lucknow ']

In [5]:
#defining lists for the data
airline_name=[]
depart_time = []
arrival_time = []
duration=[]
stops = []
source = []
destination = []
date=[]
meal=[]
price=[]
loc=[]

In [8]:
for x in city_lst:
    for y in city_lst:
        if x!=y:            
            source = driver.find_element_by_xpath("//div[@class='input-holder pb-2 bdr-btm']/input")
            time.sleep(2)
            source.clear() 
            source.send_keys(x)
            time.sleep(2)
            
            destination = driver.find_element_by_xpath("//div[@class='input-holder  bdr-btm pb-2']/input")
            time.sleep(2)
            destination.click()
            destination.clear()
            time.sleep(2)
            destination.send_keys(y)
            time.sleep(2)
            
            try:
                driver.find_element_by_xpath("/html/body/section[2]/section/div[1]/div/div[1]").click()
            except NoSuchElementException:
                pass
            try:
                srch_btn = driver.find_element_by_xpath("//button[@class='fs-14 btn-submit cursor-pointer bold']")
                wait = WebDriverWait(driver, 10)
                wait.until(EC.visibility_of(srch_btn))
                srch_btn.click()
            except NoSuchElementException:
                pass
            time.sleep(0.5)
            #try:
                #butns = driver.find_elements_by_xpath("//li[@class='i-b scroll-elem text-center']")
               # butns[1].click()
            #except NoSuchElementException:
             #   pass
            #scrolling the page
            for _ in range(15):
                time.sleep(1)
                driver.execute_script("window.scrollBy(0,5000)")
                time.sleep(5)
                #clicking on view fare
            #buttons = driver.find_elements_by_xpath("//button[@autom='morefares']")
            #for i in buttons:
                #wait = WebDriverWait(driver, 10)
                #wait.until(EC.visibility_of(i))
               # time.sleep(0.5)
               # driver.execute_script("arguments[0].click();", i)
            try:
                driver.find_element_by_xpath("//div[@title='scroll to top']").click()
            except NoSuchElementException:
                pass
           #fetching name of airline
            try:
                for i in driver.find_elements_by_xpath("//div[@class='fs-13 airline-name no-pad col-8']/span"):
                    airline_name.append(i.text)
            except NoSuchElementException:
                pass
            #time of depart
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b pr']"):
                    depart_time.append(i.text)
            except NoSuchElementException:
                pass
            #time of arrival
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b pdd-0 text-left atime col-5']//p[1]"):
                    arrival_time.append(i.get_attribute("innerHTML").splitlines()[0][0:5])
            except NoSuchElementException:
                pass
            #duration
            try:
                for i in driver.find_elements_by_xpath("//div[@class='stop-cont pl-13']/p"):
                    duration.append(i.text)
            except NoSuchElementException:
                pass
            #number of stops
            try:
                for i in driver.find_elements_by_xpath("//div[@class=' font-lightgrey fs-10 tipsy i-b fs-10']/span[1]"):
                    stops.append(i.text)
            except NoSuchElementException:
                pass
             #fetching availability of meal
            try:
                for i in driver.find_elements_by_xpath("//div[@class='features pull-right fs-12 flex']/div[1]"):
                    meal.append(i.text)
            except NoSuchElementException:
                pass
            #locations
            try:
                for i in driver.find_elements_by_xpath("//p[@class='fs-10 font-lightgrey no-wrap city ellipsis']"):
                    loc.append(i.text)
            except NoSuchElementException:
                pass
            #fetching prices
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b tipsy fare-summary-tooltip fs-18']"):
                    price.append(i.text)
            except NoSuchElementException:
                pass


In [9]:
print(len(price),len(airline_name),len(depart_time),len(arrival_time),len(duration),len(stops),len(meal),len(price))

5755 5755 5755 5755 5755 5755 5755 5755


In [10]:
source = loc[0:len(loc):2]
destination = loc[1:len(loc):2]

In [11]:
len(source), len(destination)

(5755, 5755)

In [12]:
#creating a dataframe
data = list(zip(airline_name,depart_time,arrival_time,duration,source,destination,meal,stops,price))
df = pd.DataFrame(data, columns = ["Airline","Departure_time","Time_of_arrival","Duration", "Source","Destination","Meal_availability","Number_of_stops","Price"])
df

,Airline,Departure_time,Time_of_arrival,Duration,Source,Destination,Meal_availability,Number_of_stops,Price
0,Air Asia,08:20,14:10,5h 50m,New Delhi,Mumbai,eCash 250,1 Stop,"5,949"
1,Air Asia,20:45,07:15,10h 30m,New Delhi,Mumbai,Emissions: 303 Kg CO2,1 Stop,"5,949"
2,Air Asia,09:35,20:35,11h 00m,New Delhi,Mumbai,Emissions: 303 Kg CO2,1 Stop,"5,949"
3,Air Asia,08:00,20:35,12h 35m,New Delhi,Mumbai,Emissions: 303 Kg CO2,1 Stop,"5,949"
4,SpiceJet,18:55,21:05,2h 10m,New Delhi,Mumbai,eCash 250,Non Stop,"5,953"
...,...,...,...,...,...,...,...,...,...
5750,Air India,14:45,13:15,22h 30m,Lucknow,Jaipur,Free Meal,2 Stop(s),"23,112"
5751,Air India,14:45,13:15,22h 30m,Lucknow,Jaipur,Free Meal,2 Stop(s),"23,112"
5752,Air India,14:45,13:15,22h 30m,Lucknow,Jaipur,Free Meal,2 Stop(s),"23,112"
5753,Air India,14:45,13:15,22h 30m,Lucknow,Jaipur,Free Meal,2 Stop(s),"23,112"


In [13]:
df.to_csv

<bound method NDFrame.to_csv of         Airline Departure_time Time_of_arrival Duration     Source  \
0      Air Asia          08:20           14:10   5h 50m  New Delhi   
1      Air Asia          20:45           07:15  10h 30m  New Delhi   
2      Air Asia          09:35           20:35  11h 00m  New Delhi   
3      Air Asia          08:00           20:35  12h 35m  New Delhi   
4      SpiceJet          18:55           21:05   2h 10m  New Delhi   
...         ...            ...             ...      ...        ...   
5750  Air India          14:45           13:15  22h 30m    Lucknow   
5751  Air India          14:45           13:15  22h 30m    Lucknow   
5752  Air India          14:45           13:15  22h 30m    Lucknow   
5753  Air India          14:45           13:15  22h 30m    Lucknow   
5754  Air India          14:45           13:15  22h 30m    Lucknow   

     Destination      Meal_availability Number_of_stops   Price  
0         Mumbai              eCash 250          1 Stop   5,9

In [14]:
#saving the data into csv file
df.to_csv('Flight_Price_Data.csv')